In [1]:
# IMDB - 2 layer network - train + validation + test, 20 epochs, 512 batch size (Tensorflow 2.x version, Subclassing API)
# notes: 
# - uses keras to fetch the dataset for consistancy with other code samples, data operations and feeding to network done with numpy

import tensorflow as tf

from tensorflow.keras.layers import Dense
from tensorflow.keras import Model
from tensorflow.keras.datasets import imdb

import numpy as np

#network parameters
n_input = 10000 #input size for a single sample (10000 words)

#hyperparamters
batch_size = 512
eta = 0.001 # learning rate
max_epoch = 20

# 1. get data
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

#pre-process data into tensors
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences),dimension))
    for i, sequence in enumerate(sequences):
        results[i,sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test =  vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

#validation set to use during training
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

# 2. network architecture
class IMDBModel(tf.keras.Model):
    def __init__(self):
        #constructor = define all layers (without connecting them)
        super(IMDBModel, self).__init__()
        self.fc1 = Dense(16, activation='relu')
        self.fc2 = Dense(16, activation='relu')
        self.out = Dense(1, activation='sigmoid')

    def call(self, x):
        #connect layers / tell the model the order of execution of layers
        x = self.fc1(x)
        x = self.fc2(x)
        return self.out(x)

model = IMDBModel()

# 3. select optimizer and loss
loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.RMSprop()

#define metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')

# 4. train / run network

#define training procedure
@tf.function #this makes python code compile to tensorflow C backend
def train_step(batch_x, batch_y):
    #run forward pass
    with tf.GradientTape() as tape: #gradient tape is used to "record" forward pass operations
        predictions = model(batch_x) #(1) execute all layers (reference the "call" method of the subclassed IMDBModel)
        loss = loss_object(y_true = batch_y, y_pred = predictions) #(2) calculate loss comparing labels vs. model output

    #run backpropagation, calculate and apply gradients to adjust model weights
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    #calcualte metrics
    train_loss(loss)
    train_accuracy(batch_y, predictions)

#define test procedure
@tf.function
def test_step(batch_x, batch_y):
    predictions = model(batch_x)
    t_loss = loss_object(y_true = batch_y, y_pred = predictions)

    test_loss(t_loss)
    test_accuracy(batch_y, predictions)

#run train    
for epoch in range(max_epoch):
    batch_steps = int(len(partial_x_train) / batch_size)
    for i in range(batch_steps):
        batch_x = partial_x_train[i*batch_size:(i+1)*batch_size]
        batch_y = partial_y_train[i*batch_size:(i+1)*batch_size]
        train_step(batch_x, batch_y)

    #check validation accuracy
    test_step(x_val, y_val)

    template = 'Epoch {}, loss: {} - acc: {} - val_loss: {} - val_acc: {}'
    print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result(),
                        test_loss.result(),
                        test_accuracy.result()))

    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
# 5. test model

test_loss.reset_states()
test_accuracy.reset_states()
for i in range(batch_steps):
    batch_x = x_test[i*batch_size:(i+1)*batch_size] #, tf.newaxis
    batch_y = y_test[i*batch_size:(i+1)*batch_size] #, tf.newaxis
    test_step(batch_x, batch_y)
    
print("test_acc: {}".format(test_accuracy.result()))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1, loss: 0.5347306132316589 - acc: 0.7758620977401733 - val_loss: 0.4032492935657501 - val_acc: 0.8707000017166138
Epoch 2, loss: 0.324677437543869 - acc: 0.9010640978813171 - val_loss: 0.31403589248657227 - val_acc: 0.8873000144958496
Epoch 3, loss: 0.2354554980993271 - acc: 0.9274649620056152 - val_loss: 0.28364911675453186 - val_acc: 0.8910999894142151
Epoch 4, loss: 0.18451696634292603 - acc: 0.941069

test_acc: 0.8502828478813171
